1. Петр: 30 лет, плохая кредитная история,  есть имущество для залога
2. Андрей: 42 года, цель кредита — на открытие бизнеса,  бизнес-плана нет.

Ещё раз зафиксируем основные термины:

1. **Условия перехода в ту или иную ветку называются узлами или нодами** (на диаграмме: оранжевые овалы).
2. **Узлы, которые не распадаются дальше – называют листьями** (зелёные и красные прямоугольники).
3. Узел, с которого начинает строиться дерева называют **корневым узлом** (или корнем).
Максимальная длина веток в дереве – это глубина дерева (в нашем примере: 4).

**delay_days** - дни просрочки

Наша задача обучить модель (решающее дерево), которая для новых клиентов будет предсказывать количество дней просрочки, чтобы среднеквадратичная ошибка была минимальной.

**Информационный критерий**


*   Информационный критерий — **это метод оценки качества разбиения данных на две или более группы в решающем дереве.** Это условие, по которому мы выбираем оптимальное разбиение данных в каждом узле дерева. Хороший информационный критерий помогает выбрать такой вариант разбиения выборки на две подвыборки, чтобы **ветвление дерева наиболее осмысленно разбивало данные.**



**Критерий разбиения**

*   Для задач регрессии в качестве критерия разбиения обычно используется MSE (Mean Squared Error). **MSE – это средний квадрат ошибки (разницы) между фактическими значениями целевой переменной и предсказанными. Мы считаем MSE до разбиения и после.** То разбиение, которое даст наибольшее снижение ошибки – оптимальное.

**Взвешенный критерий MSE**
- left, right – выборки слева и справа после разбиения,

- N – количество элементом выборки,

- MSE – критерий MSE.

Реализуйте две функции:

**mse** – принимает на вход вектор значений, возвращает значение mse.
**weighted_mse** – принимает на вход два вектора значений (слева и справа) и возвращает средневзвешенную mse.

**MSE**– это ошибка между истинными значениями и предсказанными. Как мы разобрали выше, предсказание в задаче регрессии – это среднее целевой переменной для всех объектов узла.

Поэтому в качестве предсказанного значения вычислите среднее значение выборки. И найдите MSE между истинными значениями выборки и предсказанным (средним).



In [ ]:
import numpy as np

def mse(y: np.ndarray) -> float:
    """Compute the mean squared error of a vector."""
    y_pred = np.mean(y)
    mse = np.mean((y_pred - y) ** 2)
    return mse


def weighted_mse(y_left: np.ndarray, y_right: np.ndarray) -> float:
    """Compute the weighted mean squared error of two vectors."""
    n_left = len(y_left)
    n_right = len(y_right)
    mse_left = mse(y_left)
    mse_right = mse(y_right)
    n_total = n_left + n_right
    mse_weight_z = (mse_left*n_left)+(mse_right*n_right)
    mse_weight = mse_weight_z / n_total
    return mse_weight

**Сплит по одному признаку**

In [ ]:
import pandas as pd
data = pd.read_csv("/content/dec_tree_2025_07_03.csv")
data

,age,income,dependents,has_property,has_car,credit_score,job_tenure,has_education,loan_amount,loan_period,delay_days
0,76,32181,3,0,1,814,28,1,142434,1770,0
1,69,52789,8,1,0,501,28,1,120887,1590,7
2,19,70535,1,0,1,325,26,1,188766,810,0
3,31,85271,1,0,1,525,29,1,406792,330,0
4,18,19974,2,0,1,618,34,1,155240,1560,43
...,...,...,...,...,...,...,...,...,...,...,...
995,66,12974,5,1,1,794,31,0,279600,480,29
996,25,65013,2,0,0,633,30,1,155824,780,0
997,24,84959,3,1,1,836,28,0,100035,750,0
998,70,37450,7,1,0,370,39,0,133343,180,1


Реализуйте **функцию split.** Функция принимает на вход выборку (матрицу признаков X и вектор целевой переменной y) и индекс признака. Возвращает значение порога с наилучшим разбиением.

Обычно берут середины подряд идущих уникальных отсортированных значений признака.
1. берем и рассчитываем порог
2. взвешенный mse

In [ ]:
from __future__ import annotations
import numpy as np

def mse(y: np.ndarray) -> float:
    """Compute the mean squared error of a vector."""
    y_pred = np.mean(y)
    mse = np.mean((y_pred - y) ** 2)
    return mse

def weighted_mse(y_left: np.ndarray, y_right: np.ndarray) -> float:
    """Compute the weighted mean squared error of two vectors."""
    n_left = len(y_left)
    n_right = len(y_right)
    mse_left = mse(y_left)
    mse_right = mse(y_right)
    n_total = n_left + n_right
    mse_weighted = (mse_left * n_left + mse_right * n_right) / n_total
    return mse_weighted

def split(X: np.ndarray, y: np.ndarray, feature: int) -> float:
    feature_values = np.unique(X[:, feature])
    best_threshold = None
    best_mse = float('inf')

    # Перебираем сами значения признака (не средние точки)
    for threshold in feature_values[:-1]:  # Все, кроме последнего
        left_mask = X[:, feature] <= threshold
        right_mask = ~left_mask

        y_left = y[left_mask]
        y_right = y[right_mask]

        if len(y_left) == 0 or len(y_right) == 0:
            continue

        current_mse = weighted_mse(y_left, y_right)

        if current_mse < best_mse:
            best_mse = current_mse
            best_threshold = threshold

    return best_threshold if best_threshold is not None else float('nan')


**Сплит по всем признакам**

Перебираем все признаки:
    Перебираем все значения признака:
        Разделяем данные по (признак, порог) -> подвыборка слева, подвыборка справа
            Считаем взвешенный критерий для подвыборок слева и справа
Возвращаем индекс признака и значение порога, которые дают минимальное значение взвешенного критерия


In [ ]:
from __future__ import annotations

import numpy as np

def split(X: np.ndarray, y: np.ndarray, feature: int) -> float:
    feature_values = np.unique(X[:, feature])
    best_threshold = None
    best_mse = float('inf')

    # Перебираем сами значения признака (не средние точки)
    for threshold in feature_values[:-1]:  # Все, кроме последнего
        left_mask = X[:, feature] <= threshold
        right_mask = ~left_mask

        y_left = y[left_mask]
        y_right = y[right_mask]

        if len(y_left) == 0 or len(y_right) == 0:
            continue

        current_mse = weighted_mse(y_left, y_right)

        if current_mse < best_mse:
            best_mse = current_mse
            best_threshold = threshold

    return best_threshold if best_threshold is not None else float('nan')

def best_split(X: np.ndarray, y: np.ndarray) -> tuple[int, float]:
    """Find the best split for a node (among all features and thresholds)"""
    n_features = X.shape[1]
    best_mse = float("inf")
    best_feature = None
    best_threshold = None

    for feature in range(n_features):
        feature_values = np.unique(X[:, feature])
        for threshold in feature_values[:-1]:  # чтобы обе части были ненулевыми
            left_mask = X[:, feature] <= threshold
            right_mask = ~left_mask

            if not np.any(left_mask) or not np.any(right_mask):
                continue  # нельзя делать сплит, если одна из частей пуста

            y_left = y[left_mask]
            y_right = y[right_mask]

            # средневзвешенное среднеквадр. отклонение
            mse = (len(y_left) * np.var(y_left, ddof=0) + len(y_right) * np.var(y_right, ddof=0)) / len(y)
            # Альтернатива — если реализована твоя weighted_mse:
            # mse = weighted_mse(y_left, y_right)

            if mse < best_mse:
                best_mse = mse
                best_feature = feature
                best_threshold = threshold

    return best_feature, best_threshold

**Node**

Давайте подумаем, какие атрибуты и каких типов необходимо добавить для класса **Node:**


Давайте подумаем, какие атрибуты и каких типов необходимо добавить для класса Node:

1. feature – индекс признака, по которому выборка, попавшая в ноду, разделяется на 2 дочерние ноды. Тип: int
2. threshold – значение порога. Все элементы, у которых значение признака feature <= threshold, попадают в левую дочернюю ноду. Остальные в правую дочернюю ноду. Тип: float
3. n_samples – количество объектов в ноде. Тип: int
4. value – среднее значение целевой переменной среди всех объектов в ноде, округленное до целого. Тип: int
5. mse – значение критерия MSE в ноде. Тип: float
6. left – левая дочерняя нода. Тип: Node
7. right – правая дочерняя нода. Тип: Node
Для листьев также будем использовать класс Node. Только поля feature, threshold, left, right не будут заполнены.

In [ ]:
from __future__ import annotations
from dataclasses import dataclass

@dataclass
class Node:
    """Decision tree node."""
    feature: int = None                      # индекс признака для сплита
    threshold: float = None                  # значение порога сплита
    n_samples: int = None                    # количество объектов в ноде
    value: int = None                        # округленное среднее целевой переменной
    mse: float = None                        # значение MSE в ноде
    left: Node = None                        # левая дочерняя нода
    right: Node = None                       # правая дочерняя нода

In [ ]:
# -*- coding: utf-8 -*-
"""Untitled48.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1ZXAso5U_ctvV24eXhKYENPG4LGqnT66F
"""

from __future__ import annotations
from dataclasses import dataclass
import numpy as np

@dataclass
class Node:
    """Decision tree node."""
    feature: int = None                      # РёРЅРґРµРєСЃ РїСЂРёР·РЅР°РєР° РґР»СЏ СЃРїР»РёС‚Р°
    threshold: float = None                  # Р·РЅР°С‡РµРЅРёРµ РїРѕСЂРѕРіР° СЃРїР»РёС‚Р°
    n_samples: int = None                    # РєРѕР»РёС‡РµСЃС‚РІРѕ РѕР±СЉРµРєС‚РѕРІ РІ РЅРѕРґРµ
    value: int = None                        # РѕРєСЂСѓРіР»РµРЅРЅРѕРµ СЃСЂРµРґРЅРµРµ С†РµР»РµРІРѕР№ РїРµСЂРµРјРµРЅРЅРѕР№
    mse: float = None                        # Р·РЅР°С‡РµРЅРёРµ MSE РІ РЅРѕРґРµ
    left: Node = None                        # Р»РµРІР°СЏ РґРѕС‡РµСЂРЅСЏСЏ РЅРѕРґР°
    right: Node = None                       # РїСЂР°РІР°СЏ РґРѕС‡РµСЂРЅСЏСЏ РЅРѕРґР°

@dataclass
class DecisionTreeRegressor:
    """Decision tree regressor."""
    max_depth: int
    min_samples_split: int = 2

    def fit(self, X: np.ndarray, y: np.ndarray) -> DecisionTreeRegressor:
        """Build a decision tree regressor from the training set (X, y)."""
        self.n_features_ = X.shape[1]
        self.tree_ = self._split_node(X, y)
        return self

    def _mse(self, y: np.ndarray) -> float:
        """Compute the mse criterion for a given set of target values."""
        if len(y) == 0:
            return 0.0
        return np.mean((y - np.mean(y)) ** 2)

    def _weighted_mse(self, y_left: np.ndarray, y_right: np.ndarray) -> float:
        """Compute the weighted mse criterion for a two given sets of target values"""
        n_left, n_right = len(y_left), len(y_right)
        n_total = n_left + n_right

        if n_total == 0:
            return 0.0

        mse_left = self._mse(y_left)
        mse_right = self._mse(y_right)

        weighted_mse = (n_left * mse_left + n_right * mse_right) / n_total
        return weighted_mse

    def _best_split(self, X: np.ndarray, y: np.ndarray) -> tuple[int, float]:
        """Find the best split for a node."""
        n_features = X.shape[1]
        best_mse = float("inf")
        best_feature = None
        best_threshold = None

        for feature in range(n_features):
            feature_values = np.unique(X[:, feature])
            for threshold in feature_values[:-1]:  # С‡С‚РѕР±С‹ РѕР±Рµ С‡Р°СЃС‚Рё Р±С‹Р»Рё РЅРµРЅСѓР»РµРІС‹РјРё
                left_mask = X[:, feature] <= threshold
                right_mask = ~left_mask

                if not np.any(left_mask) or not np.any(right_mask):
                    continue  # РЅРµР»СЊР·СЏ РґРµР»Р°С‚СЊ СЃРїР»РёС‚, РµСЃР»Рё РѕРґРЅР° РёР· С‡Р°СЃС‚РµР№ РїСѓСЃС‚Р°

                y_left = y[left_mask]
                y_right = y[right_mask]

                mse = self._weighted_mse(y_left, y_right)

                if mse < best_mse:
                    best_mse = mse
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold

    def _split_node(self, X: np.ndarray, y: np.ndarray, depth: int = 0) -> Node:
        """Split a node and return the resulting left and right child nodes."""
        node = Node()
        node.n_samples = len(y)
        node.value = int(round(np.mean(y)))
        node.mse = self._mse(y)

        # Check stopping criteria
        if depth >= self.max_depth or len(y) < self.min_samples_split:
            return node

        # Find best split
        feature, threshold = self._best_split(X, y)

        # If no split found (all splits are bad)
        if feature is None or threshold is None:
            return node

        # Split the data
        left_mask = X[:, feature] <= threshold
        right_mask = ~left_mask

        # Create child nodes
        node.feature = feature
        node.threshold = threshold
        node.left = self._split_node(X[left_mask], y[left_mask], depth + 1)
        node.right = self._split_node(X[right_mask], y[right_mask], depth + 1)

        return node

DecisionTreeRegressor -> **Depth-wise подход**
Подведем промежуточный итог, чему мы уже научились:

1. Считать информационный критерий и взвешенный критерий, чтобы оценить качество сплита.
2. Находить наилучшее разбиение выборки.
3. Определили сущность нода и описали ее классом Node.

In [ ]:
data

,age,income,dependents,has_property,has_car,credit_score,job_tenure,has_education,loan_amount,loan_period,delay_days
0,76,32181,3,0,1,814,28,1,142434,1770,0
1,69,52789,8,1,0,501,28,1,120887,1590,7
2,19,70535,1,0,1,325,26,1,188766,810,0
3,31,85271,1,0,1,525,29,1,406792,330,0
4,18,19974,2,0,1,618,34,1,155240,1560,43
...,...,...,...,...,...,...,...,...,...,...,...
995,66,12974,5,1,1,794,31,0,279600,480,29
996,25,65013,2,0,0,633,30,1,155824,780,0
997,24,84959,3,1,1,836,28,0,100035,750,0
998,70,37450,7,1,0,370,39,0,133343,180,1


In [ ]:
from __future__ import annotations
from dataclasses import dataclass
import numpy as np
import json
from typing import Dict, Any

@dataclass
class Node:
    """Decision tree node."""
    feature: int = None
    threshold: float = None
    n_samples: int = None
    value: int = None
    mse: float = None
    left: Node = None
    right: Node = None

@dataclass
class DecisionTreeRegressor:
    """Decision tree regressor."""
    max_depth: int
    min_samples_split: int = 2

    def fit(self, X: np.ndarray, y: np.ndarray) -> DecisionTreeRegressor:
        """Build a decision tree regressor from the training set (X, y)."""
        self.n_features_ = X.shape[1]
        self.tree_ = self._split_node(X, y)
        return self

    def _mse(self, y: np.ndarray) -> float:
        """Compute the mse criterion for a given set of target values."""
        if len(y) == 0:
            return 0.0
        y_pred = np.mean(y)
        return np.mean((y_pred - y) ** 2)

    def _weighted_mse(self, y_left: np.ndarray, y_right: np.ndarray) -> float:
        """Compute the weighted mse criterion for a two given sets of target values"""
        n_left = len(y_left)
        n_right = len(y_right)
        mse_left = self._mse(y_left)
        mse_right = self._mse(y_right)
        n_total = n_left + n_right
        return (mse_left * n_left + mse_right * n_right) / n_total

    def _best_split(self, X: np.ndarray, y: np.ndarray) -> tuple[int, float]:
        """Find the best split for a node."""
        best_mse = float("inf")
        best_feature, best_threshold = None, None

        for feature in range(self.n_features_):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds[:-1]:
                left_mask = X[:, feature] <= threshold
                right_mask = ~left_mask

                if sum(left_mask) < self.min_samples_split or sum(right_mask) < self.min_samples_split:
                    continue

                current_mse = self._weighted_mse(y[left_mask], y[right_mask])
                if current_mse < best_mse:
                    best_mse = current_mse
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold


    def _split_node(self, X: np.ndarray, y: np.ndarray, depth: int = 0) -> Node:
        """Split a node and return the resulting left and right child nodes."""
        node = Node()
        node.n_samples = len(y)
        node.value = int(round(np.mean(y)))
        node.mse = self._mse(y)

        if (depth >= self.max_depth or
            len(y) < self.min_samples_split or
            len(np.unique(y)) == 1):
            return node

        feature, threshold = self._best_split(X, y)
        if feature is None:
            return node

        left_mask = X[:, feature] <= threshold
        right_mask = ~left_mask

        node.feature = feature
        node.threshold = threshold
        node.left = self._split_node(X[left_mask], y[left_mask], depth + 1)
        node.right = self._split_node(X[right_mask], y[right_mask], depth + 1)

        return node

    def as_json(self) -> str:
        """Return the decision tree as a JSON string."""
        if not hasattr(self, 'tree_'):
            return "{}"
        tree_dict = self._tree_to_dict(self.tree_)
        return json.dumps(tree_dict)

    def _as_json(self, node: Node) -> str:
        """Return the decision tree as a JSON string. Execute recursively."""
        if node.left is None and node.right is None:
            # Листовой узел
            return {
                "value": node.value,
                "n_samples": node.n_samples,
                "mse": round(node.mse, 2) if node.mse is not None else None
            }
        else:
            # Внутренний узел
            return {
                "feature": node.feature,
                "threshold": node.threshold,
                "n_samples": node.n_samples,
                "mse": round(node.mse, 2) if node.mse is not None else None,
                "left": self._tree_to_dict(node.left),
                "right": self._tree_to_dict(node.right)
            }

In [ ]:
data

,age,income,dependents,has_property,has_car,credit_score,job_tenure,has_education,loan_amount,loan_period
0,76,32181,3,0,1,814,28,1,142434,1770
1,69,52789,8,1,0,501,28,1,120887,1590
2,19,70535,1,0,1,325,26,1,188766,810
3,31,85271,1,0,1,525,29,1,406792,330
4,18,19974,2,0,1,618,34,1,155240,1560
...,...,...,...,...,...,...,...,...,...,...
995,66,12974,5,1,1,794,31,0,279600,480
996,25,65013,2,0,0,633,30,1,155824,780
997,24,84959,3,1,1,836,28,0,100035,750
998,70,37450,7,1,0,370,39,0,133343,180


In [37]:
from __future__ import annotations
from dataclasses import dataclass
import numpy as np
import json
from typing import Dict, Any

@dataclass
class Node:
    """Decision tree node."""
    feature: int = None
    threshold: float = None
    n_samples: int = None
    value: int = None
    mse: float = None
    left: Node = None
    right: Node = None

@dataclass
class DecisionTreeRegressor:
    """Decision tree regressor."""
    max_depth: int
    min_samples_split: int = 2

    def fit(self, X: np.ndarray, y: np.ndarray) -> DecisionTreeRegressor:
        """Build a decision tree regressor from the training set (X, y)."""
        self.n_features_ = X.shape[1]
        self.tree_ = self._split_node(X, y)
        return self

    def _mse(self, y: np.ndarray) -> float:
        """Compute the mse criterion for a given set of target values."""
        if len(y) == 0:
            return 0.0
        return np.mean((y - np.mean(y)) ** 2)

    def _weighted_mse(self, y_left: np.ndarray, y_right: np.ndarray) -> float:
        """Compute the weighted mse criterion for a two given sets of target values"""
        n_left, n_right = len(y_left), len(y_right)
        n_total = n_left + n_right

        if n_total == 0:
            return 0.0

        mse_left = self._mse(y_left)
        mse_right = self._mse(y_right)

        weighted_mse = (n_left * mse_left + n_right * mse_right) / n_total
        return weighted_mse

    def _best_split(self, X: np.ndarray, y: np.ndarray) -> tuple[int, float]:
        """Find the best split for a node."""
        n_features = X.shape[1]
        best_mse = float("inf")
        best_feature = None
        best_threshold = None

        for feature in range(n_features):
            unique_values = np.unique(X[:, feature])
            for threshold in unique_values[:-1]:
                left_mask = X[:, feature] <= threshold
                right_mask = ~left_mask

                if np.sum(left_mask) == 0 or np.sum(right_mask) == 0:
                    continue

                y_left = y[left_mask]
                y_right = y[right_mask]

                current_mse = self._weighted_mse(y_left, y_right)

                if current_mse < best_mse:
                    best_mse = current_mse
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold

    def _split_node(self, X: np.ndarray, y: np.ndarray, depth: int = 0) -> Node:
        """Split a node and return the resulting left and right child nodes."""
        node = Node()
        node.n_samples = int(len(y))
        node.value = int(round(np.mean(y)))
        node.mse = self._mse(y)

        # Check stopping criteria
        if depth >= self.max_depth or len(y) < self.min_samples_split:
            return node

        # Find best split
        feature, threshold = self._best_split(X, y)

        # If no split found (all splits are bad)
        if feature is None or threshold is None:
            return node

        # Split the data
        left_mask = X[:, feature] <= threshold
        right_mask = ~left_mask

        # Create child nodes
        node.feature = int(feature)
        node.threshold = int(threshold)
        node.left = self._split_node(X[left_mask], y[left_mask], depth + 1)
        node.right = self._split_node(X[right_mask], y[right_mask], depth + 1)

        return node

    def as_json(self) -> str:
        """Return the decision tree as a JSON string."""
        if not hasattr(self, 'tree_'):
            raise ValueError("The tree hasn't been trained yet. Call fit() first.")
        import json
        return json.dumps(self._as_json(self.tree_), indent=2)

    def _as_json(self, node: Node) -> dict:
        """Return the decision tree node as a dictionary (part of JSON)."""
        if node.left is None and node.right is None:
            # Leaf node
            return {
                "value": node.value,
                "n_samples": node.n_samples,
                "mse": round(node.mse, 2) if node.mse is not None else None
            }
        else:
            # Internal node
            return {
                "feature": node.feature,
                "threshold": node.threshold,
                "n_samples": node.n_samples,
                "mse": round(node.mse, 2) if node.mse is not None else None,
                "left": self._as_json(node.left),
                "right": self._as_json(node.right)
            }

    def predict(self, X: np.ndarray) -> np.ndarray:
        """Predict regression target for X."""
        if not hasattr(self, 'tree_'):
            raise ValueError("The tree hasn't been trained yet. Call fit() first.")
        return np.array([self._predict_one_sample(x) for x in X])

    def _predict_one_sample(self, features: np.ndarray) -> int:
        """Predict the target value of a single sample."""
        node = self.tree_

        while node.left is not None and node.right is not None:
            if features[node.feature] <= node.threshold:
                node = node.left
            else:
                node = node.right

        return int(node.value) if node.value is not None else None

In [42]:
import pandas as pd
data = pd.read_csv("/content/dec_tree_2025_07_09.csv")

y = data.pop("delay_days")
X = data

X_test = X.values
y_test = y.values

depth = 3

user_tree = DecisionTreeRegressor(max_depth=depth)
user_tree.n_features_ = X.shape[1]

user_tree.fit(X_test, y_test)
preds = user_tree.predict(X_test)
df_compare = pd.DataFrame({
    'real': y_test,
    'pred': preds
})
print(df_compare.head(10))

   real  pred
0     0    38
1     7    10
2     0     2
3     0     2
4    43    38
5     0     2
6     3    14
7     0     2
8     0     2
9    12    40
